#Install package

In [ ]:
!pip install bitsandbytes==0.43.1
!pip install accelerate==0.30.1
!pip install transformers==4.39.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

#Huggingface Login

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

#Load Llama3 with Quantization

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
# Specify the model ID for the pre-trained model from Hugging Face
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# Load the tokenizer using the specified model ID
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Configure the BitsAndBytes settings for loading the model
config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Load the model in 4-bit precision to save memory and possibly improve performance
    bnb_4bit_quant_type="nf4",            # Use "nf4" as the quantization type for 4-bit precision
    bnb_4bit_use_double_quant=True,       # Enable double quantization with quantization constant
    bnb_4bit_compute_dtype=torch.bfloat16 # Use bfloat16 as the computation data type to balance speed and precision
)

# Load the model for causal language modeling using the specified model ID
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",         # torch_dtype="auto" automatically selects the appropriate data type for the model
    device_map="auto",          # device_map="auto" automatically maps the model to available devices (like GPU if available)
    quantization_config=config  # quantization_config=config specifies the quantization configuration for the model
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
model.num_parameters()

8030261248

#Inference with Single

In [ ]:
# Define the prompt
messages = [
    {"role": "system", "content": "You are a kind robot."},
    {"role": "user", "content": "이순신이 누구야?"},
]

# Tokenize the prompt
input_ids = tokenizer.apply_chat_template(
    messages2,
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=True
).to(model.device)

print(input_ids)

tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,   3169,
          12585,     13,   1472,  15832,   4320,    304,  16526,     13, 128009,
         128006,    882, 128007,    271,  42529, 102757,  67945, 108668,  13094,
         123621,  90759,     30, 128009, 128006,  78191, 128007,    271]],
       device='cuda:0')


In [ ]:
# Define a list of terminators, which specifies the end-of-sequence token ID
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
# Generate model outputs based on the input IDs
outputs = model.generate(
    input_ids=input_ids,                 # Provide the tokenized input prompt
    max_new_tokens=300,                  # Specify the maximum number of new tokens to generate
    eos_token_id=terminators,            # Specify the end-of-sequence token ID to stop generation
    do_sample=True,                      # Enable stochastic mode (randomness)
    temperature=0.7,                     # Control the randomness of predictions. Higher values make the output more random, lower values make it more deterministic.
    no_repeat_ngram_size=2,              # Prevent repetition of 2-gram sequences. This option prevents the model from repeating the same sequence of 2 tokens (words) more than once.
    pad_token_id=tokenizer.eos_token_id  # Use the end-of-sequence token as the padding token.
)

# Extract the generated response tokens, excluding the input prompt tokens
response = outputs[0][input_ids.shape[-1]:]

# Decode the response tokens back into a string, skipping special tokens
print("response : ", tokenizer.decode(response, skip_special_tokens=True))


response :  ! (Sejong the Great was the fourth king of the Joseon Dynasty in Korea. He is known for his efforts to promote education and culture, and is considered one of Korea's most important monarchs.)


#Inference with Batch

In [ ]:
# Define the prompt
messages1 = [
    {"role": "system", "content": "You are a kind robot."},
    {"role": "user", "content": "이순신이 누구야?"},
]

prompt1 = tokenizer.apply_chat_template(
    messages1,
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=False
)

print(prompt1)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a kind robot.<|eot_id|><|start_header_id|>user<|end_header_id|>

이순신이 누구야?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [ ]:
messages2 = [
    {"role": "system", "content": "You are a kind robot."},
    {"role": "user", "content": "세종대왕이 누구야?"},
]

prompt2 = tokenizer.apply_chat_template(
    messages2,
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=False
)

print(prompt2)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a kind robot.<|eot_id|><|start_header_id|>user<|end_header_id|>

세종대왕이 누구야?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [ ]:
tokenizer.padding_side = 'left'
tokenizer.pad_token_id = tokenizer.eos_token_id

prompt_batch = [prompt1, prompt2]
input_ids_batch = tokenizer(prompt_batch, return_tensors='pt',padding="longest")['input_ids']

In [ ]:
# Define a list of terminators, which specifies the end-of-sequence token ID
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids=input_ids_batch,           # Provide the tokenized input prompt
    max_new_tokens=30,                  # Specify the maximum number of new tokens to generate
    eos_token_id=terminators,            # Specify the end-of-sequence token ID to stop generation
    do_sample=True,                      # Enable stochastic mode (randomness)
    temperature=0.7,                     # Control the randomness of predictions. Higher values make the output more random, lower values make it more deterministic.
    no_repeat_ngram_size=2,              # Prevent repetition of 2-gram sequences. This option prevents the model from repeating the same sequence of 2 tokens (words) more than once.
    pad_token_id=tokenizer.eos_token_id  # Use the end-of-sequence token as the padding token.
)


In [ ]:
# 각 배치에 대한 응답 추출 및 디코딩
for i, output in enumerate(outputs):
    response = output[input_ids_batch[i].shape[-1]:]
    print(f"response {i + 1}: ", tokenizer.decode(response, skip_special_tokens=True))

response 1:  I think you're asking who Yi Sun-sin is!

Yi Sun-shin (1545-1598) was a Korean admiral and
response 2:  I'm happy to help! 😊

Sejong the Great (1396-1450) was the fourth king of the Joseon Dynasty in


#Compare Latency

In [ ]:
import time
from tqdm.auto import tqdm

def measure_latency(input_ids, num_iterations=2, max_new_tokens=30):
    """
    Measure the latency of the model generation process.

    Args:
    - input_ids: The tokenized input prompt.
    - num_iterations: The number of iterations to measure latency.
    - max_new_tokens: The maximum number of new tokens to generate.

    Returns:
    - Average latency over the specified number of iterations.
    """
    latencies = []

    for i in tqdm(range(num_iterations)):
        start_time = time.time()
        outputs = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            eos_token_id=terminators,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
        latencies.append(time.time() - start_time)

    average_latency = sum(latencies) / len(latencies)
    return average_latency

In [ ]:
# Define the prompt
messages = [
    {"role": "system", "content": "You are a kind robot."*30},
    {"role": "user", "content": "Do know Yi Sun-sin?"},
]

# Tokenize the prompt
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# Example usage:
average_latency = measure_latency(input_ids)
print(f"Average latency: {average_latency:.4f} seconds")


  0%|          | 0/2 [00:00<?, ?it/s]

Average latency: 14.7434 seconds


In [ ]:
# Define the prompt
messages = [
    {"role": "system", "content": "You are a kind robot."*30},
    {"role": "user", "content": "Do know Yi Sun-sin?"},
]

# Tokenize the prompt
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# Example usage:
average_latency = measure_latency(input_ids)
print(f"Average latency: {average_latency:.4f} seconds")

  0%|          | 0/2 [00:00<?, ?it/s]

Average latency: 58.6016 seconds
